In [44]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf


from doctr.io import DocumentFile
from doctr.models import ocr_predictor

In [45]:
doc = DocumentFile.from_pdf("test2.pdf")
print(f"Number of pages: {len(doc)}")

Number of pages: 3


In [46]:
predictor_ocr_model = ocr_predictor(pretrained=True)

In [47]:
ocr_result = predictor_ocr_model(doc)

In [48]:
ocr_text = ocr_result.render()
ocr_text

'SCHAEFFLER\n\nOriginal\n\nSchaeffler Technologies AG & Co. KG 91072 Herzogenaurach (Germany)\n\nSchaeffler India Limited\nPlant 0295 / FBI (CPLM)\nHead Office & Works\nw/p Maneja\nVADODARA 390013\nINDIA\n\nPage:\n1/3\n\nInvoice\nDocument No./ Date\nOrder No. Date\nPurch. Order Date\nPurch. Order No.\nOur Vendor Number\nYour Reference\nSales Group\nCustomer\nCurrency\nInvoiced Amount\n\n135511380 I 19.01.2024\n613854972/ 16.01.2024\n16.01.2024\n0036047645\n0000006066\nVirendrakumar K.\nIndia\n23864\nINR\n991,467.40\n\nDelivery Note No. Date 26213921 / 19.01.2024\n\nShip-to address\nSchaeffler India Limited\nPlant 0042/1 IBI (CPLM) Components\nVillage Ambi\nPlot A-3\nPUNE 410507\nINDIA\nUnloading point: 77777\nCustomer\nSchaeffler India Limited\nPlant 0042/1 IBI (CPLM) Components\nVillage Ambi\nPlot A-3\nPUNE 410507\nINDIA\n\nResponsible employee Santosh Pol\n\nYour VAT Registration No. 24AAACF3357Q12C\nOur VAT Registration No. DE291636029\n\nVAT: Tax free exportation (Article 146 Counc

In [70]:
import os
from urllib import response as url_response
import requests

from groq import Groq

client = Groq(
    api_key="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",
)

error_message = "Not able to create"  

# Prompt for the API call
prompt = """Extract entities and their values from the provided OCR text "ocr_text", and separate them by a new line. Provide output by adding a : bewteen the entites and their value. Make sure to use all the data from the "ocr_text".  \nText:{ocr_text}\nEntities:"""

# Send the prompt to Groq API for completion
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": f"error_message:{error_message}, ocr_text:{ocr_text}",

        },
    ],
    model="llama3-70b-8192",
)

# Get the response content
response_content = chat_completion.choices[0].message.content

print(response_content)


Here are the extracted entities with their values, separated by a newline and colon:

**Company**: Schaeffler Technologies AG & Co. KG

**Address**: 91072 Herzogenaurach (Germany)

**Invoice Number**: 135511380

**Invoice Date**: 19.01.2024

**Order Number**: 613854972

**Order Date**: 16.01.2024

**Purchaser Order Number**: 0036047645

**Purchaser Order Date**: 16.01.2024

**Vendor Number**: 0000006066

**Sales Group**: Virendrakumar K.

**Customer**: India

**Currency**: INR

**Invoiced Amount**: 991,467.40

**Delivery Note Number**: 26213921

**Delivery Date**: 19.01.2024

**Ship-to Address**: Schaeffler India Limited, Plant 0042/1 IBI (CPLM) Components, Village Ambi, Plot A-3, PUNE 410507, INDIA

**Responsible Employee**: Santosh Pol

**VAT Registration Number**: 24AAACF3357Q12C, DE291636029

**Material Number**: 005281938-0000-12, 000235660-0063-10

**Item Description**: F-46954-3511.BZ.SLH#N Bolt, NRB4,5X9,8-G2-2-4#N Needle roller

**Order Quantity**: 96,000 PC, 301,000 PC

**Uni

In [71]:
import os
from urllib import response as url_response

from groq import Groq

client = Groq(
    # This is the default and can be omitted
    api_key="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",
)

entites = response_content

error_message = "Not able to create"

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
                    "content": """
                    You have been provided with the extracted OCR Data "ocr_text" and the entites of these text in the "entites" list for an invoice.
                    Create a JSON object from the information present in this OCR Data "ocr_text" and the entites of these text in the "entites" list.
                    Please keep in mind the "entites" and the OCR Data. This is because the "entites" list contains the extracted entities and their values from the OCR Data.
                    Keep in mind the semantic meaning of the ocr_text.
                    Make sure your response is a dict and not a list.
                    {% if error_message %}
                    You already created the following output in a previous attempt: {{invalid_replies}}
                    However, this doesn't comply with the format requirements from above and triggered this Python exception: "error_message"
                    Correct the output and try again. Just return the corrected output without any extra explanations.
                    If no answer value is found for a certain key, then simply fill in "NA"
                    {% endif %}
                    Your final output should be a JSON object that contains the properties of the invoice. This data would be used for a Supplier Invoice Automation system.
                    """
        },
        {
            "role": "user",
            "content": f"error_message:{error_message}, ocr_text:{ocr_text}, entites:{entites}"
        }
    ],
    model="llama3-70b-8192",
)

response_content = chat_completion.choices[0].message.content


In [72]:
print(response_content)

Here is the JSON object based on the provided OCR data and entities:

```
{
    "invoice": {
        "company": "Schaeffler Technologies AG & Co. KG",
        "address": "91072 Herzogenaurach (Germany)",
        "invoice_number": "135511380",
        "invoice_date": "19.01.2024",
        "order_number": "613854972",
        "order_date": "16.01.2024",
        "purchaser_order_number": "0036047645",
        "purchaser_order_date": "16.01.2024",
        "vendor_number": "0000006066",
        "sales_group": "Virendrakumar K.",
        "customer": "India",
        "currency": "INR",
        "invoiced_amount": 991467.40
    },
    "delivery": {
        "delivery_note_number": "26213921",
        "delivery_date": "19.01.2024",
        "ship_to_address": "Schaeffler India Limited, Plant 0042/1 IBI (CPLM) Components, Village Ambi, Plot A-3, PUNE 410507, INDIA",
        "responsible_employee": "Santosh Pol"
    },
    "vat_registration_number": {
        "seller": "24AAACF3357Q12C",
        "bu

In [73]:
import json

def remove_na_values(obj):
    if isinstance(obj, dict):
        return {k: remove_na_values(v) for k, v in obj.items() if v != "NA" and v is not None}
    elif isinstance(obj, list):
        return [remove_na_values(item) for item in obj if item != "NA" and item is not None]
    else:
        return obj

copy_response = response_content
json_start_index = copy_response.find("{")
json_end_index = copy_response.rfind("}")
data = json.loads(copy_response[json_start_index:json_end_index + 1])


cleaned_data = remove_na_values(data)
cleaned_json = json.dumps(cleaned_data, indent=2)
print(cleaned_json)

{
  "invoice": {
    "company": "Schaeffler Technologies AG & Co. KG",
    "address": "91072 Herzogenaurach (Germany)",
    "invoice_number": "135511380",
    "invoice_date": "19.01.2024",
    "order_number": "613854972",
    "order_date": "16.01.2024",
    "purchaser_order_number": "0036047645",
    "purchaser_order_date": "16.01.2024",
    "vendor_number": "0000006066",
    "sales_group": "Virendrakumar K.",
    "customer": "India",
    "currency": "INR",
    "invoiced_amount": 991467.4
  },
  "delivery": {
    "delivery_note_number": "26213921",
    "delivery_date": "19.01.2024",
    "ship_to_address": "Schaeffler India Limited, Plant 0042/1 IBI (CPLM) Components, Village Ambi, Plot A-3, PUNE 410507, INDIA",
    "responsible_employee": "Santosh Pol"
  },
  "vat_registration_number": {
    "seller": "24AAACF3357Q12C",
    "buyer": "DE291636029"
  },
  "items": [
    {
      "material_number": "005281938-0000-12",
      "item_description": "F-46954-3511.BZ.SLH#N Bolt",
      "order_qu